# Model Selection with WBIC

In [11]:
from datetime import timedelta

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import orbit
print(orbit.__version__)
from orbit.models import DLT,ETS, KTRLite, LGT
from orbit.utils.simulation import make_trend, make_regression

1.1.0dev


In [12]:
orbit.__version__

'1.1.0dev'

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Generate a regression problem with trend with `8` number of regressors where only `3` of them are effective. First, generate the `3` effective regressors.

In [14]:
NUM_OF_REGRESSORS = 8
NUM_OF_EFFECTIVE_REGRESSORS = 3
SERIES_LEN = 100
SEED = 20210101
# sample some coefficients
COEFS = np.random.default_rng(SEED).uniform(-1, 1, NUM_OF_EFFECTIVE_REGRESSORS)
trend = make_trend(SERIES_LEN, rw_loc=0.01, rw_scale=0.1)
x, regression, coefs = make_regression(series_len=SERIES_LEN, coefs=COEFS)
print(regression.shape, x.shape)

# combine trend and the regression
y = trend + regression
y = y - y.min()

(100,) (100, 3)


In [15]:
# add missing data 
missing_idx = np.array([10, 20, 30, 40, 41, 42, 43, 44, 100 - 1])
y[missing_idx] = np.nan


We can add `5` irrelevant regressors into the dataset to add challenge in selecting the best model.

In [16]:
x_extra = np.random.normal(0, 1, (SERIES_LEN, NUM_OF_REGRESSORS - NUM_OF_EFFECTIVE_REGRESSORS))
x = np.concatenate([x, x_extra], axis=-1)
print(x.shape)

(100, 8)


In [17]:
x_cols = [f"x{x}" for x in range(1, NUM_OF_REGRESSORS + 1)]
response_col = "y"
dt_col = "date"
obs_matrix = np.concatenate([y.reshape(-1, 1), x], axis=1)
# make a data frame for orbit inputs
df = pd.DataFrame(obs_matrix, columns=[response_col] + x_cols)
# make some dummy date stamp
dt = pd.date_range(start='2016-01-04', periods=SERIES_LEN, freq="1W")
df['date'] = dt
print(df.shape)
print(df.head())

(100, 10)
          y        x1        x2        x3        x4        x5        x6  \
0  3.010526  0.172792  0.000000  0.165219 -0.893116 -1.274101 -0.061154   
1  3.448303 -0.000000  0.452678  0.223187 -0.572882 -0.801334  1.312035   
2  3.012242 -0.000000  0.290559  0.182286  0.313719 -1.444821 -0.368961   
3  1.422250  0.147066  0.014211  0.273356  0.057294  2.089979  0.041971   
4  3.275717 -0.368227 -0.081455 -0.241060 -0.084589 -1.215450 -1.412931   

         x7        x8       date  
0  0.064514  0.410113 2016-01-10  
1  1.274699 -1.214358 2016-01-17  
2 -0.769227  0.392616 2016-01-24  
3 -0.048341 -0.513154 2016-01-31  
4 -1.486911  0.382225 2016-02-07  


In [18]:
 regressor_col = x_cols[:3 + 1]


In [19]:
dlt_mod = DLT(
        response_col=response_col,
        date_col=dt_col,
        regressor_col=regressor_col,
        seed=2020,
        # fixing the smoothing parameters to learn regression coefficients more effectively
        level_sm_input=0.01,
        slope_sm_input=0.01,
        num_warmup=4000,
        num_sample=4000,

    )
dlt_mod.fit_WBIC(df=df) #, , sampling_temperature = np.log(100.0)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_41c61aa86230eb6107141a4d10977c03 NOW.
In file included from /var/folders/2r/w1lk3nxj09s73plknkyzzj5m0000gn/T/pystan_qjyfzp8v/stanfit4anon_model_41c61aa86230eb6107141a4d10977c03_7568374438624539410.cpp:771:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969:
/Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it wit

/Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/src/Core/AssignEvaluator.h:879:5: note: in instantiation of function template specialization 'Eigen::internal::call_dense_assignment_loop<Eigen::Matrix<double, -1, -1, 0>, Eigen::Product<Eigen::Matrix<double, -1, -1, 0>, Eigen::Transpose<const Eigen::Matrix<double, -1, -1, 0>>, 1>, Eigen::internal::assign_op<double, double>>' requested here
    call_dense_assignment_loop(dst, src, func);
    ^
/Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/src/Core/AssignEvaluator.h:836:46: note: in instantiation of member function 'Eigen::internal::Assignment<Eigen::Matrix<double, -1, -1, 0>, Eigen::Product<Eigen::Matrix<double, -1, -1, 0>, Eigen::Transpose<const Eigen::Matrix<double, -1, -1, 0>>, 1>, Eigen::internal::assign_op<double, double>, Eigen::internal::Dense2Dense, void>::run' requested here
  Ass

In file included from /var/folders/2r/w1lk3nxj09s73plknkyzzj5m0000gn/T/pystan_qjyfzp8v/stanfit4anon_model_41c61aa86230eb6107141a4d10977c03_7568374438624539410.cpp:780:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/py_var_context.hpp:12:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/src/stan/io/dump.hpp:6:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:9:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat/meta/as_array_or_scalar.hpp:4:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Dense:1:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages

In file included from /var/folders/2r/w1lk3nxj09s73plknkyzzj5m0000gn/T/pystan_qjyfzp8v/stanfit4anon_model_41c61aa86230eb6107141a4d10977c03_7568374438624539410.cpp:781:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan_fit.hpp:15:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/lib/boost_1.69.0/boost/random/additive_combine.hpp:27:
/Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/lib/boost_1.69.0/boost/random/linear_congruential.hpp:140:20: warning: overlapping comparisons always evaluate to false [-Wtautological-overlap-compare]
        if(_x <= 0 && _x != 0) {
           ~~~~~~~~^~~~~~~~~~
/Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/lib/boost_1.69.0/boost/random/linear_congruential.hpp:393:11: note: in instantiation of member function 'boost::ran

In file included from /var/folders/2r/w1lk3nxj09s73plknkyzzj5m0000gn/T/pystan_qjyfzp8v/stanfit4anon_model_41c61aa86230eb6107141a4d10977c03_7568374438624539410.cpp:780:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/py_var_context.hpp:12:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/src/stan/io/dump.hpp:6:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:9:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat/meta/as_array_or_scalar.hpp:4:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Dense:1:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages

...

In file included from /var/folders/2r/w1lk3nxj09s73plknkyzzj5m0000gn/T/pystan_qjyfzp8v/stanfit4anon_model_41c61aa86230eb6107141a4d10977c03_7568374438624539410.cpp:780:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/py_var_context.hpp:12:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/src/stan/io/dump.hpp:6:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:9:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat/meta/as_array_or_scalar.hpp:4:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Dense:1:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages

In file included from /var/folders/2r/w1lk3nxj09s73plknkyzzj5m0000gn/T/pystan_qjyfzp8v/stanfit4anon_model_41c61aa86230eb6107141a4d10977c03_7568374438624539410.cpp:780:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/py_var_context.hpp:12:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/src/stan/io/dump.hpp:6:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:9:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat/meta/as_array_or_scalar.hpp:4:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Dense:1:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages

156 warnings generated.



Gradient evaluation took 0.000218 seconds
1000 transitions using 10 leapfrog steps per transition would take 2.18 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.00026 seconds
1000 transitions using 10 leapfrog steps per transition would take 2.6 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.00022 seconds
1000 transitions using 10 leapfrog steps per transition would take 2.2 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.000276 seconds
1000 transitions using 10 leapfrog steps per transition would take 2.76 seconds.
Adjust your expectations accordingly!


Iteration:    1 / 2000 [  0%]  (Warmup)
Iteration:    1 / 2000 [  0%]  (Warmup)
Iteration:    1 / 2000 [  0%]  (Warmup)
Iteration:    1 / 2000 [  0%]  (Warmup)
Iteration:  200 / 2000 [ 10%]  (Warmup)
Iteration:  400 / 2000 [ 20%]  (Warmup)
Iteration:  200 / 2000 [ 10%]  (Warmup)
Iteration:  600 / 2000 [ 30%]  (Warmup)
Iteration:  200 / 2000 [ 10%

-129.1832165371954

In [ ]:
ets_mod = ETS(
        response_col=response_col,
        date_col=dt_col,
        seed=2020,
        # fixing the smoothing parameters to learn regression coefficients more effectively
        level_sm_input=0.01,
        num_warmup=4000,
        num_sample=4000,

    )

ets_mod.fit_WBIC(df)

In [ ]:
KTRLite_mod = KTRLite(
        response_col=response_col,
        date_col=dt_col,
        seed=2020,
        # fixing the smoothing parameters to learn regression coefficients more effectively
        #level_sm_input=0.01,
        estimator='stan-map',
        #num_warmup=4000,
        #num_sample=4000,

    )

KTRLite_mod.fit(df)

In [23]:
LGT_mod = LGT(
        response_col=response_col,
        date_col=dt_col,
        seed=2020,
        # fixing the smoothing parameters to learn regression coefficients more effectively
        #level_sm_input=0.01,
        estimator='stan-mcmc',
        num_warmup=4000,
        num_sample=4000,

    )

Q = LGT_mod.fit(df)


Gradient evaluation took 0.000427 seconds
1000 transitions using 10 leapfrog steps per transition would take 4.27 seconds.
Adjust your expectations accordingly!


Iteration:    1 / 2000 [  0%]  (Warmup)

Gradient evaluation took 0.00026 seconds
1000 transitions using 10 leapfrog steps per transition would take 2.6 seconds.
Adjust your expectations accordingly!


Iteration:    1 / 2000 [  0%]  (Warmup)

Gradient evaluation took 0.000271 seconds
1000 transitions using 10 leapfrog steps per transition would take 2.71 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.000217 seconds
1000 transitions using 10 leapfrog steps per transition would take 2.17 seconds.
Adjust your expectations accordingly!


Iteration:    1 / 2000 [  0%]  (Warmup)
Iteration:    1 / 2000 [  0%]  (Warmup)
Iteration:  200 / 2000 [ 10%]  (Warmup)
Iteration:  200 / 2000 [ 10%]  (Warmup)
Iteration:  200 / 2000 [ 10%]  (Warmup)
Iteration:  200 / 2000 [ 10%]  (Warmup)
Iteration:  400 / 2000 [ 2

['l', 'b', 'lev_sm', 'slp_sm', 'obs_sigma', 'nu', 'lgt_sum', 'gt_pow', 'lt_coef', 'gt_coef']
odict_keys(['l', 'b', 'lev_sm', 'slp_sm', 'obs_sigma', 'nu', 'lgt_sum', 'gt_pow', 'lt_coef', 'gt_coef'])


In [24]:
from orbit.estimators.stan_estimator import StanEstimatorMCMC, StanEstimatorMAP


def test_stan_estimator_mcmc(stan_estimator_lgt_model_input):
    stan_model_name, model_param_names, data_input = stan_estimator_lgt_model_input

    # create estimator
    mcmc_estimator = StanEstimatorMCMC(num_warmup=50)

    # extract posterior samples
    posteriors, training_metrics = mcmc_estimator.fit(
        model_name=stan_model_name, model_param_names=model_param_names, data_input=data_input)

    print('left hand side')
    print(model_param_names)
    print('right hand side')
    print(posteriors.keys())
    
    assert set(model_param_names) == set(posteriors.keys())


In [26]:
test_stan_estimator_mcmc(,LGT_mod.get_training_data_input())

ValueError: too many values to unpack (expected 3)

In [27]:
LGT_mod.get_training_data_input()

{'RESPONSE': array([3.01052592, 3.44830335, 3.01224196, 1.42225017, 3.27571742,
        3.15745992, 1.97805118, 3.09564191, 3.26676413, 2.98030022,
               nan, 3.73767771, 2.18748214, 2.78466386, 2.36271233,
        3.30554298, 3.08325817, 2.29709297, 2.25523723, 2.39774258,
               nan, 2.11881671, 4.13159968, 3.78031361, 0.        ,
        0.86234144, 2.57033959, 1.96729669, 2.67672453, 2.86319904,
               nan, 0.80583112, 1.86643967, 5.28191483, 3.52440418,
        3.44419058, 2.29480558, 1.45839575, 2.83451337, 3.36402938,
               nan,        nan,        nan,        nan,        nan,
        2.61011093, 2.51651567, 2.38019022, 3.35978123, 3.41408492,
        2.92418901, 2.4173399 , 2.74935528, 2.20140491, 3.81879859,
        1.64602044, 3.68107555, 2.95883051, 0.98088538, 2.58858296,
        2.66679532, 2.69640431, 1.74470337, 1.89167753, 3.34656278,
        1.77997273, 3.60926157, 4.1082015 , 0.60491735, 3.26345619,
        4.01732871, 2.03346856, 1.70

In [ ]:
LGT_mod = LGT(
        response_col=response_col,
        date_col=dt_col,
        seed=2020,
        # fixing the smoothing parameters to learn regression coefficients more effectively
        #level_sm_input=0.01,
        estimator='stan-map',
        num_warmup=4000,
        num_sample=4000,

    )

LGT_mod.fit(df)

In [ ]:
dlt_mod = DLT(
        response_col=response_col,
        date_col=dt_col,
        regressor_col=regressor_col,
        seed=2020,
        # fixing the smoothing parameters to learn regression coefficients more effectively
        level_sm_input=0.01,
        slope_sm_input=0.01,
        num_warmup=4000,
        num_sample=4000,
        estimator='stan-map',

    )
dlt_mod.fit(df=df) #, , sampling_temperature = np.log(100.0)

In [ ]:
ets_mod.fit_WBIC(df=df)

In [ ]:
ets_mod = ETS(
        response_col=response_col,
        date_col=dt_col,
        seed=2020,
        # fixing the smoothing parameters to learn regression coefficients more effectively
        level_sm_input=0.01,
        num_warmup=4000,
        num_sample=4000,

    )
ets_mod.fit(df=df,sampling_temperature = np.log(100.0)) #, , sampling_temperature = np.log(100.0)

In [ ]:
ets_mod.get_WBIC_value()

In [ ]:
ForecasterException: Model class: <class 'orbit.template.dlt.DLTModel'> is incompatible with 
        Estimator: <class 'orbit.estimators.stan_estimator.StanEstimatorMCMC'>.  
            Estimator Support: [<class 'orbit.estimators.stan_estimator.StanEstimatorMAP'>, 
                                <class 'orbit.estimators.stan_estimator.StanEstimatorMCMC'>
                                <class 'orbit.estimators.stan_estimator.StanEstimatorMCMC'>]

Now, we can calculate WBIC and compare them across models.

In [ ]:
%%time
wbics = np.empty(NUM_OF_REGRESSORS)

for idx in range(NUM_OF_REGRESSORS):
    regressor_col = x_cols[:idx + 1]

    dlt_mod = DLT(
        response_col=response_col,
        date_col=dt_col,
        regressor_col=regressor_col,
        seed=2020,
        # fixing the smoothing parameters to learn regression coefficients more effectively
        level_sm_input=0.01,
        slope_sm_input=0.01,
        num_warmup=4000,
        num_sample=4000,
    )
    dlt_mod.fit(df=df)
    wbic = dlt.get_training_metrics()['WBIC']
    print('Regressors:{} WBIC:{:.5f}'.format(regressor_col, wbic))
    wbics[idx] = wbic

We plot the chart with WBICs against number of regressors included.  As we can see, WBIC is lowest when regressors overlapped exactly with the truth.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))
ax.plot(np.arange(1, NUM_OF_REGRESSORS + 1), wbics, color='dodgerblue', label='WBICs')
ax.axvline(x=3, linestyle='--', color='orange', label='truth')
ax.set_xlabel('Number of Regressors')
ax.set_ylabel('WBIC')
ax.set_title('Model Selection with WBIC')
fig.legend()